<a href="https://colab.research.google.com/github/Vimalkumar-Narasimman/AI/blob/main/CellPhone_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

In [ ]:
!kaggle datasets download -d vitorgamalemos/cellphone

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!unzip dogs-vs-cats.zip -d drive/MyDrive/Data

In [ ]:
!unzip cellphone.zip -d drive/MyDrive/Data/cellphone

In [ ]:
import os
import zipfile
import random
import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model

In [ ]:
files_trainNO = 'drive/MyDrive/Data/cellphone/training/training/cellphone-NO/'
files_trainYES = 'drive/MyDrive/Data/cellphone/training/training/cellphone-YES/'
files_OS_trainNO = os.listdir(files_trainNO)
files_OS_trainYES = os.listdir(files_trainYES)
trainingSamples = np.size(files_OS_trainNO) + np.size(files_OS_trainYES)
print(' No cellphone samples:', np.size(files_OS_trainNO))
print(' Yes cellphone samples:', np.size(files_OS_trainYES))
print(' Total training samples:', trainingSamples)

In [ ]:
# Function to load and preprocess images
def load_and_preprocess_images(directory, img_size=(80, 80)):
    images = []
    labels = []
    category = directory.split('/')[-2].split('-')[-1]
    print(category)
    for filename in os.listdir(directory):
        img_array = cv2.imread(os.path.join(directory, filename), cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=img_size)
        images.append(new_img_array)
        labels.append(1 if 'yes' in category.lower() else 0)  # Assuming 'yes' is positive class, 'no' is negative

    return np.array(images), np.array(labels)

# Load and preprocess images
imagesNo, labelsNo = load_and_preprocess_images(files_trainNO, img_size=(80, 80))
imagesYes, labelsYes = load_and_preprocess_images(files_trainYES, img_size=(80, 80))

In [ ]:
# Display 8 random No images after labeling
plt.figure(figsize=(15, 10))
for i in range(8):
    random_index = random.randint(0, len(imagesNo)-1)
    plt.subplot(2, 4, i+1)
    plt.imshow(imagesNo[random_index], cmap='gray')
    plt.title(f"Label: {labelsNo[random_index]}")
    plt.axis('off')

plt.show()

In [ ]:
# Display 8 random Yes images after labeling
plt.figure(figsize=(15, 10))
for i in range(8):
    random_index = random.randint(0, len(imagesYes)-1)
    plt.subplot(2, 4, i+1)
    plt.imshow(imagesYes[random_index], cmap='gray')
    plt.title(f"Label: {labelsYes[random_index]}")
    plt.axis('off')

plt.show()

In [ ]:

images =  np.concatenate((imagesNo, imagesYes), axis=0)
labels =  np.concatenate((labelsNo, labelsYes), axis=0)
images, labels = shuffle(images, labels, random_state=2)

In [ ]:
# Display 8 random images after labeling
plt.figure(figsize=(15, 10))
for i in range(8):
    random_index = random.randint(0, len(images)-1)
    plt.subplot(2, 4, i+1)
    plt.imshow(images[random_index], cmap='gray')
    plt.title(f"Label: {labels[random_index]}")
    plt.axis('off')

plt.show()

In [ ]:
# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to be between 0 and 1
train_images, val_images = train_images / 255.0, val_images / 255.0

In [ ]:
# Display 8 random images after labeling
plt.figure(figsize=(15, 10))
for i in range(8):
    random_index = random.randint(0, len(val_images)-1)
    plt.subplot(2, 4, i+1)
    plt.imshow(val_images[random_index], cmap='gray')
    plt.title(f"Label: {val_labels[random_index]}")
    plt.axis('off')

plt.show()

In [ ]:
# Create an ImageDataGenerator with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a generator for both training and validation data with augmentation
train_generator = train_datagen.flow(
    train_images.reshape((-1, 80, 80, 1)),
    train_labels,
    batch_size=32
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow(
    val_images.reshape((-1, 80, 80, 1)),
    val_labels,
    batch_size=32
)

In [ ]:
# Display 8 random Yes images after labeling
plt.figure(figsize=(15, 10))
for i in range(8):
    random_index = random.randint(0, len(train_generator)-1)
    plt.subplot(2, 4, i+1)
    plt.imshow(train_generator[random_index][0][0], cmap='gray')
    plt.title(f"Label: {train_generator[random_index][1][0]}")
    plt.axis('off')

plt.show()

In [ ]:
# Define the revised model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 1)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=binary_crossentropy, metrics=['accuracy'])

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with the generator for both training and validation
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
    #,callbacks=[early_stopping]
)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Predict classes for validation set
y_pred = (model.predict(val_generator) > 0.5).astype("int32")

# Compute confusion matrix
cm = confusion_matrix(val_labels, y_pred)

# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.title('Confusion Matrix')
plt.show()

In [ ]:
precision = precision_score(val_labels, y_pred)
recall = recall_score(val_labels, y_pred)
f1 = f1_score(val_labels, y_pred)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
# Function to load and preprocess test images
def load_and_preprocess_test_images(directory, img_size=(80, 80)):
    images = []

    for filename in os.listdir(directory):
        img_array = cv2.imread(os.path.join(directory, filename), cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=img_size)
        images.append(new_img_array)

    return np.array(images)

# Directory containing test images
test_directory =  'drive/MyDrive/Data/cellphone/test_samples/test_samples/'

# Load and preprocess test images
test_images = load_and_preprocess_test_images(test_directory, img_size=(80, 80))
test_images = shuffle(test_images, random_state=2)
# Reshape test images to match the model's input shape
test_images1 = test_images.reshape((-1, 80, 80, 1))



# Use the best model to make predictions on the test data
test_predictions = model.predict(test_images1)

In [ ]:
# Display some predicted images
def display_predicted_images(images, predictions, num_images=4):
    plt.figure(figsize=(15, 5))
    startImg = 20
    for i in range(startImg,startImg+num_images):
        plt.subplot(1, num_images, i-startImg + 1)
        plt.imshow(images[i].reshape((80, 80)), cmap='gray')
        plt.title(f'Predicted: {predictions[i][0]}')
        plt.axis('off')
    plt.show()

# Display predicted images with actual labels
display_predicted_images(test_images, test_predictions, num_images=6)